In [9]:
import pandas as pd
import sys
import importlib
import fasttext

base_dir = '/Users/anasputhawala/Desktop/Winterproj/'
sys.path.insert(0, base_dir)

from src.utils import pre_processing
from src.models.tfidf import config_tfidf
import fasttext_prep
importlib.reload(pre_processing)
importlib.reload(config_tfidf)
importlib.reload(fasttext_prep)

<module 'fasttext_prep' from '/Users/anasputhawala/Desktop/Winterproj/src/models/fasttext/fasttext_prep.py'>

In [10]:
fasttext_prep.load_prep_save()

Train and Test .txt files have successfully been saved at /Users/anasputhawala/Desktop/Winterproj/src/models/fasttext


In [41]:
train_path = '/Users/anasputhawala/Desktop/Winterproj/src/models/fasttext/train.txt'
test_path = '/Users/anasputhawala/Desktop/Winterproj/src/models/fasttext/test.txt'
model_ft = fasttext.train_supervised(input=train_path, wordNgrams=3, epoch=5, lr=0.15)

Read 1M words
Number of words:  88486
Number of labels: 16
Progress: 100.0% words/sec/thread:  910919 lr:  0.000000 avg.loss:  2.199809 ETA:   0h 0m 0s


In [42]:
model_ft.test(test_path)

(54420, 0.2984196986402058, 0.2984196986402058)